In this notebook we write some tests while implementing the new deepmod version.

In [1]:
import numpy as np
import pandas as pd
import torch

from DeePyMoD_SBL.data import Burgers
from DeePyMoD_SBL.deepymod_torch.library_functions import library_1D_in
from DeePyMoD_SBL.deepymod_torch.DeepMod import DeepMod, DeepModDynamic
from DeePyMoD_SBL.deepymod_torch.output import Tensorboard, progress
from DeePyMoD_SBL.deepymod_torch.losses import reg_loss, mse_loss
from DeePyMoD_SBL.deepymod_torch.training import train
from DeePyMoD_SBL.Lasso import lasso
from DeePyMoD_SBL.deepymod_torch.sparsity import scaling

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

import matplotlib.pyplot as plt


import time


%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

# Making data

In [29]:
x = np.linspace(-2, 5, 75)
t = np.linspace(0.5, 5.0, 25)

x_grid, t_grid = np.meshgrid(x, t, indexing='ij')

In [30]:
dataset = Burgers(0.1, 1.0)

In [31]:
u = dataset.solution(x_grid, t_grid)

In [32]:
X = np.concatenate((t_grid.reshape(-1, 1), x_grid.reshape(-1, 1)), axis=1)
y = u.reshape(-1, 1)

In [33]:
X_train = torch.tensor(X, dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y, dtype=torch.float32, requires_grad=True)

In [34]:
noise_level = 0.05

In [35]:
X_train = torch.tensor(X, dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y + noise_level * np.std(y) * np.random.randn(y.size, 1), dtype=torch.float32, requires_grad=True)

# Running MSE and PI only with new method

In [36]:
config = {'n_in': 2, 'hidden_dims': [30, 30, 30, 30, 30], 'n_out': 1, 'library_function':library_1D_in, 'library_args':{'poly_order':2, 'diff_order': 2}, 'fit_method': 'lstsq'}

In [37]:
model = DeepModDynamic(**config)

In [38]:
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True)

In [ ]:
%%time
ata = X_train
target = y_train
max_iterations = 5000
loss_func_args={'l1':torch.tensor(1e-5), 'conv_tol': torch.tensor(1e-3)}


print('| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |')
for iteration in torch.arange(0, max_iterations + 1):
    # Calculating prediction and library and scaling
    prediction, time_deriv_list, sparse_theta_list, coeff_vector_list, theta = model(data)
    coeff_vector_scaled_list = scaling(coeff_vector_list, sparse_theta_list, time_deriv_list)
        
    # Calculating loss
    loss_reg = reg_loss(time_deriv_list, sparse_theta_list, coeff_vector_list)
    loss_mse = mse_loss(prediction, target)
    loss = torch.sum(loss_reg) + torch.sum(loss_mse)
        
    # Writing
    if iteration % 100 == 0:
        progress(iteration, 0.0, max_iterations, loss.item(), torch.sum(loss_mse).item(), torch.sum(loss_reg).item(), 0.0)
           
    # Optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |
       3100     62.00%               0s   2.11e-01   1.37e-01   7.38e-02   0.00e+00 

In [ ]:
model.fit.coeff_vector

# Running MSE and PI only with old method

In [47]:
config = {'n_in': 2, 'hidden_dims': [30, 30, 30, 30, 30], 'n_out': 1, 'library_function':library_1D_in, 'library_args':{'poly_order':2, 'diff_order': 2}, 'fit_method': 'optim'}

In [48]:
model = DeepModDynamic(**config)

In [49]:
optimizer = torch.optim.Adam(model.network_parameters(), betas=(0.99, 0.999), amsgrad=True)

In [50]:
%%time
data = X_train
target = y_train
max_iterations = 5000
loss_func_args={'l1':torch.tensor(1e-5), 'conv_tol': torch.tensor(1e-3)}


print('| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |')
for iteration in torch.arange(0, max_iterations + 1):
    # Calculating prediction and library and scaling
    prediction, time_deriv_list, sparse_theta_list, coeff_vector_list, theta = model(data)
    coeff_vector_scaled_list = scaling(coeff_vector_list, sparse_theta_list, time_deriv_list)
        
    # Calculating loss
    loss_reg = reg_loss(time_deriv_list, sparse_theta_list, coeff_vector_list)
    loss_mse = mse_loss(prediction, target)
    loss = torch.sum(loss_reg) + torch.sum(loss_mse)
        
    # Writing
    if iteration % 100 == 0:
        progress(iteration, 0.0, max_iterations, loss.item(), torch.sum(loss_mse).item(), torch.sum(loss_reg).item(), 0.0)
           
    # Optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |
       5000    100.00%               0s   1.44e-01   1.21e-01   2.23e-02   0.00e+00 CPU times: user 55.6 s, sys: 437 ms, total: 56 s
Wall time: 56.2 s


In [46]:
model.fit.coeff_vector[0]

Parameter containing:
tensor([[0.9753],
        [0.7733],
        [0.7434],
        [0.4163],
        [0.5839],
        [0.0178],
        [0.9313],
        [0.0829],
        [0.7103]], requires_grad=True)